# Modelo de Recomendación Item-Item:


El modelo debe establecer una relación ítem-ítem, donde la recomendación de elementos similares se basa en la medida de similitud entre un ítem dado y el resto. En este contexto, el modelo toma como entrada un juego específico y genera como salida una lista de juegos recomendados. Para lograr esto, se recomienda aplicar la métrica de similitud del coseno.

Para lo anterior se hace uso de los datos guardados en la carpeta "Clean_Data" sobre el archivo "SGames_CD.csv" donde desde el inicio se permitió que los datos en genero quedaran en listas sin extender porque podría facilitar el desarrollo del modelo. Aunque no se descarta la opción de extender los generos.

### 1. Importar las librerías necesarias para la creación del modelo y descarga de datos:

In [1]:
import pandas as pd                                                 # Para abrir y editar los archivos 
import numpy as np                                                  # Para realizar cálculos

#----------------------------------------------------------------------------------------------------------------------------------

from sklearn.feature_extraction.text import CountVectorizer         # Para convertir las características analizadas en matrices
from sklearn.metrics.pairwise import cosine_similarity              # Para calcular la similitud de coseno de la matriz


### 2. Abrir y arreglar los datos para que no hayan registros nulos:

In [2]:
Model_Data = pd.read_csv('Clean_Data\\SGames_CD.csv', quotechar='"')
Model_Data.head(2)

,publisher,genres,app_name,release_date,specs,price,early_access,id,developer,release_year
0,Kotoshiro,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Lost Summoner Kitty,2018-01-04,['Single-player'],4.99,False,761140.0,Kotoshiro,2018
1,"Making Fun, Inc.","['Free to Play', 'Indie', 'RPG', 'Strategy']",Ironbound,2018-01-04,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,643980.0,Secret Level SRL,2018


In [3]:
Model_Data=Model_Data[['id','genres','publisher','release_date','specs','price','early_access','developer','release_year']]
Model_Data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32133 entries, 0 to 32132
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            32132 non-null  float64
 1   genres        31994 non-null  object 
 2   publisher     24062 non-null  object 
 3   release_date  29781 non-null  object 
 4   specs         31463 non-null  object 
 5   price         30267 non-null  float64
 6   early_access  32133 non-null  bool   
 7   developer     28834 non-null  object 
 8   release_year  32133 non-null  int64  
dtypes: bool(1), float64(2), int64(1), object(5)
memory usage: 2.0+ MB


Se realizan los cambios para garantizar que no hay datos nulos, se eliminan datos que tengan al mismo tiempo nulos en año y genero, para los demás valores se imputa con desconocido:

In [4]:
# Eliminar registros con valores nulos en 'genres' y valores de 0 en 'release_year'

Model_Data = Model_Data[(Model_Data['release_year'] != 0) & (Model_Data['genres'].notnull())]

# Reemplazar valores nulos en 'genres' con 'desconocido'
Model_Data['genres'] = Model_Data['genres'].fillna('a stranger')

# Reemplazar valores nulos en 'developer' y 'publisher' con 'desconocido':
Model_Data['developer'] = Model_Data['developer'].fillna('a stranger')
Model_Data['publisher'] = Model_Data['publisher'].fillna('a stranger')

# Colocar 0 en los valores nulos de 'price'
Model_Data['price'] = Model_Data['price'].fillna(0)

# Eliminar registros con valores nulos en 'genres'
Model_Data = Model_Data.dropna(subset=['genres'])

# Eliminar la columna 'release_date'
Model_Data = Model_Data.drop(columns=['release_date'])

# Colocar 'not specified' en los valores nulos de la columna 'specs'
Model_Data['specs'] = Model_Data['specs'].fillna('not specified')

# Verificar los cambios
Model_Data.head(-5)


,id,genres,publisher,specs,price,early_access,developer,release_year
0,761140.0,"['Action', 'Casual', 'Indie', 'Simulation', 'S...",Kotoshiro,['Single-player'],4.99,False,Kotoshiro,2018
1,643980.0,"['Free to Play', 'Indie', 'RPG', 'Strategy']","Making Fun, Inc.","['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Secret Level SRL,2018
2,670290.0,"['Casual', 'Free to Play', 'Indie', 'Simulatio...",Poolians.com,"['Single-player', 'Multi-player', 'Online Mult...",0.00,False,Poolians.com,2017
3,767400.0,"['Action', 'Adventure', 'Casual']",彼岸领域,['Single-player'],0.99,False,彼岸领域,2017
5,772540.0,"['Action', 'Adventure', 'Simulation']",Trickjump Games Ltd,"['Single-player', 'Steam Achievements']",3.99,False,Trickjump Games Ltd,2018
...,...,...,...,...,...,...,...,...
32122,761480.0,"['Adventure', 'Indie']",Phil Fortier,"['Single-player', 'Steam Achievements']",0.99,False,Phil Fortier,2018
32123,771810.0,"['Action', 'Adventure', 'Indie']",Retro Army Limited,"['Single-player', 'Captions available']",0.00,False,Retro Army Limited,2018
32124,767590.0,"['Casual', 'Indie']",OrtiGames/OrtiSoft,"['Single-player', 'Shared/Split Screen', 'Stea...",0.99,False,"Oscar Ortigueira López,OrtiGames/OrtiSoft",2018
32125,747320.0,"['Indie', 'RPG']",INGAME,"['Single-player', 'Steam Achievements', 'Steam...",14.99,False,INGAME,2018


Se obtiene una base sin nulos para trabajar:

In [5]:
Model_Data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 29644 entries, 0 to 32131
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   id            29644 non-null  float64
 1   genres        29644 non-null  object 
 2   publisher     29644 non-null  object 
 3   specs         29644 non-null  object 
 4   price         29644 non-null  float64
 5   early_access  29644 non-null  bool   
 6   developer     29644 non-null  object 
 7   release_year  29644 non-null  int64  
dtypes: bool(1), float64(2), int64(1), object(4)
memory usage: 1.8+ MB


In [6]:
# Contar registros con valor 0 en 'release_year'
count_zero_release_year = (Model_Data['release_year'] == 0).sum()

# Imprimir el resultado
print("Cantidad de registros con release_year igual a 0:", count_zero_release_year)


Cantidad de registros con release_year igual a 0: 0


### 3. Desarrollo del modelo "Similitud del Coseno":

3.1. Convertir "genres" y "specs" en cadena para facilitar el desarrollo del modelo:

In [7]:
# Eliminar comillas, corchetes, comas y reemplazar guiones por espacios en 'genres'
Model_Data['genres'] = Model_Data['genres'].astype(str).str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")

# Eliminar comillas, corchetes, comas y reemplazar guiones por espacios en 'specs'
Model_Data['specs'] = Model_Data['specs'].astype(str).str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")

# Verificar los cambios
Model_Data.head()


,id,genres,publisher,specs,price,early_access,developer,release_year
0,761140.0,Action Casual Indie Simulation Strategy,Kotoshiro,Single player,4.99,False,Kotoshiro,2018
1,643980.0,Free to Play Indie RPG Strategy,"Making Fun, Inc.",Single player Multi player Online Multi Player...,0.00,False,Secret Level SRL,2018
2,670290.0,Casual Free to Play Indie Simulation Sports,Poolians.com,Single player Multi player Online Multi Player...,0.00,False,Poolians.com,2017
3,767400.0,Action Adventure Casual,彼岸领域,Single player,0.99,False,彼岸领域,2017
5,772540.0,Action Adventure Simulation,Trickjump Games Ltd,Single player Steam Achievements,3.99,False,Trickjump Games Ltd,2018


3.2. Tokenizar genres y specs para facilitar al modelo el análisis (no se exige en estos modelos pero parece apropiado) también se normaliza. Para developer y Publisher se decide codificar:

In [8]:
# Tokenizar y normalizar 'genres'
Model_Data['genres'] = Model_Data['genres'].astype(str).str.lower().str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")

# Tokenizar y normalizar 'specs'
Model_Data['specs'] = Model_Data['specs'].astype(str).str.lower().str.replace('[', '').str.replace(']', '').str.replace("'", "").str.replace(",", "").str.replace("-", " ")


In [24]:
# Codificación de dummies para 'publisher' y 'developer'
#Model_Data = pd.get_dummies(Model_Data, columns=['publisher', 'developer'], drop_first=True)


# Verificar los cambios
Model_Data.head()

,id,genres,publisher,specs,price,early_access,developer,release_year,combined_features
0,761140.0,action casual indie simulation strategy,Kotoshiro,single player,4.99,False,Kotoshiro,2018,action casual indie simulation strategy single...
1,643980.0,free to play indie rpg strategy,"Making Fun, Inc.",single player multi player online multi player...,0.00,False,Secret Level SRL,2018,free to play indie rpg strategy single player ...
2,670290.0,casual free to play indie simulation sports,Poolians.com,single player multi player online multi player...,0.00,False,Poolians.com,2017,casual free to play indie simulation sports si...
3,767400.0,action adventure casual,彼岸领域,single player,0.99,False,彼岸领域,2017,action adventure casual single player
5,772540.0,action adventure simulation,Trickjump Games Ltd,single player steam achievements,3.99,False,Trickjump Games Ltd,2018,action adventure simulation single player stea...


3.3. Exportar dataframe a CSV para que sea insumo del modelo, aunque lo más probable es que solo se requiera la matriz de similitud para el modelo:

In [11]:
# Exportar el DataFrame a CSV:

#Model_Data.to_csv('Data_Queries/Model_Data.csv', index=False)
#print(f"DataFrame exportado a {'Data_Queries/Model_Data.csv'}")

#Acción suspendida, dado que el df no fué util en la api. Se procede a crear un df arreglado para facilitar las funciones

DataFrame exportado a Data_Queries/Model_Data.csv


3.3. Aplicar el modelo de similitud de coseno:

In [11]:
# Combina las columnas 'genres' y 'specs' en una sola columna de texto
Model_Data['combined_features'] = Model_Data['genres'] + ' ' + Model_Data['specs']

# Selecciona las características relevantes (One-Hot Encoded + combined_features)
text_features = pd.get_dummies(Model_Data['combined_features'])
encoded_features = pd.get_dummies(Model_Data[['publisher', 'developer']], prefix=['publisher', 'developer'])
features = pd.concat([encoded_features, text_features], axis=1)

# Utiliza CountVectorizer para convertir la columna 'combined_features' en una matriz de recuentos
vectorizer = CountVectorizer()
text_features_matrix = vectorizer.fit_transform(Model_Data['combined_features'])

# Combina la matriz de recuentos con las características relevantes
all_features_matrix = pd.concat([pd.DataFrame(text_features_matrix.toarray()), features.reset_index(drop=True)], axis=1)

# Calcula la similitud de coseno entre los vectores
cosine_sim = cosine_similarity(all_features_matrix, all_features_matrix)

# Imprime la matriz de similitud de coseno
print(cosine_sim)

[[1.         0.31192515 0.41105415 ... 0.34503278 0.32659863 0.35082321]
 [0.31192515 1.         0.67161876 ... 0.4663724  0.50937163 0.45596075]
 [0.41105415 0.67161876 1.         ... 0.28365431 0.28767798 0.30901572]
 ...
 [0.34503278 0.4663724  0.28365431 ... 1.         0.56343617 0.72627304]
 [0.32659863 0.50937163 0.28767798 ... 0.56343617 1.         0.5728919 ]
 [0.35082321 0.45596075 0.30901572 ... 0.72627304 0.5728919  1.        ]]


3.4. Crear una función de similitudo de coseno para iterar en la base de datos:

In [34]:

def similitud (id):

# Elegir un ítem específico (cambiar 'selected_item_id' por el ID deseado)
    selected_item_id = 	id

# Obtener la fila correspondiente al ítem seleccionado
    selected_item_index = Model_Data[Model_Data['id'] == selected_item_id].index[0]

# Calcular la similitud de coseno entre el ítem seleccionado y todos los demás ítems
    cosine_similarities = cosine_sim[selected_item_index]

# Obtener los índices de los ítems más similares (excluyendo el propio ítem seleccionado)
    similar_items_indices = np.argsort(cosine_similarities)[::-1][1:6]

# Obtener los IDs de los ítems más similares
    similar_item_ids = Model_Data.loc[similar_items_indices, 'id'].tolist()

    return id,similar_item_ids


3.5. Probar función:

In [35]:
similitud(772540)

(772540, [768880.0, 774276.0, 774277.0, 773650.0, 237740.0])

3.6. Iterar en la base de datos, para obtener para cada id los juesgos recomendados:

In [41]:
dic_similitud = {}

for i in Model_Data['id']:
    try:
        value = similitud(i)
        if value == i:
            continue
        else:
            dic_similitud[i] = value
    except Exception as e:
        print(f"Error para el ID {i}: {str(e)}")

Error para el ID 761140.0: '[14232] not in index'
Error para el ID 643980.0: '[19795] not in index'
Error para el ID 774278.0: '[3372] not in index'
Error para el ID 775881.0: '[6665] not in index'
Error para el ID 752360.0: '[7425] not in index'
Error para el ID 766850.0: '[22] not in index'
Error para el ID 773120.0: '[11308] not in index'
Error para el ID 754360.0: '[12464] not in index'
Error para el ID 759920.0: '[13224] not in index'
Error para el ID 764280.0: '[45] not in index'
Error para el ID 4230.0: '[26860] not in index'
Error para el ID 6510.0: '[13676] not in index'
Error para el ID 7650.0: '[74] not in index'
Error para el ID 2300.0: '[20742] not in index'
Error para el ID 4580.0: '[13507] not in index'
Error para el ID 4760.0: '[26062] not in index'
Error para el ID 12500.0: '[13513, 10428] not in index'
Error para el ID 16180.0: '[51, 26062] not in index'
Error para el ID 15970.0: '[45] not in index'
Error para el ID 15930.0: '[6554] not in index'
Error para el ID 1524

3.7. Crear dataframe del diciconario obtenido:

In [42]:
# Crear listas para almacenar datos
ids = []
recommended_ids = []

# Iterar sobre el diccionario y crear las listas
for key, values in dic_similitud.items():
    ids.extend([key] * len(values))
    recommended_ids.extend(values)

# Crear un DataFrame a partir de las listas
similitud_df = pd.DataFrame({'id': ids, 'recommended_id': recommended_ids})

# Imprimir el DataFrame resultante
similitud_df

,id,recommended_id
0,670290.0,670290.0
1,670290.0,"[614300.0, 605490.0, 608590.0, 505750.0, 49295..."
2,767400.0,767400.0
3,767400.0,"[720850.0, 72500.0, 464525.0, 45302.0, 484860.0]"
4,772540.0,772540.0
...,...,...
36533,263340.0,"[680340.0, 349760.0, 717190.0, 461780.0, 35734..."
36534,263380.0,263380.0
36535,263380.0,"[271860.0, 762350.0, 439910.0, 639530.0, 73511..."
36536,264320.0,264320.0


In [44]:
# Filtrar solo las filas donde 'recommended_id' es una lista
similitud_df = similitud_df[similitud_df['recommended_id'].apply(lambda x: isinstance(x, list))]

# Restablecer los índices después de la filtración
similitud_df.reset_index(drop=True, inplace=True)

# Imprimir el DataFrame resultante
similitud_df


,id,recommended_id
0,670290.0,"[614300.0, 605490.0, 608590.0, 505750.0, 49295..."
1,767400.0,"[720850.0, 72500.0, 464525.0, 45302.0, 484860.0]"
2,772540.0,"[768880.0, 774276.0, 774277.0, 773650.0, 23774..."
3,774276.0,"[774276.0, 774277.0, 768880.0, 772540.0, 23774..."
4,774277.0,"[774276.0, 774277.0, 768880.0, 772540.0, 23774..."
...,...,...
18264,267652.0,"[271474.0, 256611.0, 376956.0, 572700.0, 50549..."
18265,267650.0,"[289881.0, 260550.0, 433450.0, 287700.0, 31173..."
18266,263340.0,"[680340.0, 349760.0, 717190.0, 461780.0, 35734..."
18267,263380.0,"[271860.0, 762350.0, 439910.0, 639530.0, 73511..."


3.8. Creamos tablas de nombres

In [48]:
names = pd.read_csv('Clean_Data\\SGames_CD.csv', quotechar='"')
names=names[['id','app_name']]

3.9. Exportamos nuevo dataframe con id y los id recomendados por similitud de coseno y tabla de nombres:

In [64]:
# Exportar el DataFrame a CSV:

similitud_df.to_csv('Data_Queries/similitud_df.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/similitud_df.csv'}")

names.to_csv('Data_Queries/names.csv', index=False)
print(f"DataFrame exportado a {'Data_Queries/names.csv'}")

DataFrame exportado a Data_Queries/similitud_df.csv
DataFrame exportado a Data_Queries/names.csv


3.10. Crear función para la Fastapi:

In [69]:
def obtener_nombres_recomendados(id_seleccionado):
    try:
        # Obtener el nombre del juego seleccionado
        nombre_seleccionado = names.loc[names['id'] == id_seleccionado, 'app_name'].iloc[0]

        # Obtener los IDs de los juegos recomendados
        juegos_recomendados_ids = similitud_df.loc[similitud_df['id'] == id_seleccionado, 'recommended_id'].iloc[0]

        # Obtener los nombres de los juegos recomendados
        nombres_recomendados = []
        for id_recomendado in juegos_recomendados_ids:
            nombre_recomendado = names.loc[names['id'] == id_recomendado, 'app_name'].iloc[0]
            nombres_recomendados.append(nombre_recomendado)

        # Crear y devolver el diccionario
        diccionario_resultado = {'nombre_seleccionado': nombre_seleccionado, 'nombres_recomendados': nombres_recomendados}
        return diccionario_resultado

    except IndexError:
        print(f"No se encontró nombre para el ID seleccionado {id_seleccionado}")

    except KeyError:
        print(f"No se encontraron recomendaciones para el ID seleccionado {id_seleccionado}")

    except Exception as e:
        print(f"Error inesperado: {str(e)}")


3.11. Probar función:

In [70]:
# Ejemplo de uso
id_seleccionado = 670290  # Ingresa el ID deseado
resultado = obtener_nombres_recomendados(id_seleccionado)
print(resultado)

{'nombre_seleccionado': 'Real Pool 3D - Poolians', 'nombres_recomendados': ['Fantasy Grounds - Mini-Dungeon #023: The Aura of Profit (5E)', 'One Hit KO', 'Hack_me 2 - Wallpapers', 'Trimmer Tycoon', 'Rocksmith® 2014 – 2010s Mix Song Pack']}
